## 第3章例3.5.1 投入产出价格模型

In [175]:
from gamspy import Container, Set, Parameter, Variable, Equation, Model, Sum, Alias, Domain, Sense
from gamspy.math import log
import pandas as pd
# pd.options.display.float_format = '{:.1f}'.format 

### data

In [176]:
data = pd.read_excel("ch3-5-1.xlsx", index_col=0)
data

,sec1,sec2,sec3,consumption,investment,Total
sec1,200,300,150,280.0,70.0,1000.0
sec2,80,400,250,550.0,320.0,1600.0
sec3,30,420,240,350.0,110.0,1150.0
labor,500,250,330,NaN,NaN,NaN
capital,190,230,180,NaN,NaN,NaN
Total,1000,1600,1150,NaN,NaN,NaN


### sets

In [177]:
m = Container()
ac = Set(m, name="ac", records=data.index[:-1].append(data.columns[3:]).values)
acc = Alias(m, name="acc", alias_with=ac)
i = Set(m, name="i", domain=ac, records=data.index.values[0:3])
j = Alias(m, name="j", alias_with=i)
x = Set(m, name="x", domain=ac, records=data.index.values[3:5])
s = Set(m, name="s", domain=[i], records=["sec1","sec3"])
# ss = Alias(m, name="ss", alias_with=s)

### parameter

In [178]:
iotable = Parameter(m, name="iotable", description="original data", domain=[ac,acc],records=data.stack().reset_index())
iotable.records.pivot(index="level_0",columns="level_1",values="value")

level_1,sec1,sec2,sec3,consumption,investment,Total
level_0,,,,,,
sec1,200.0,300.0,150.0,280.0,70.0,1000.0
sec2,80.0,400.0,250.0,550.0,320.0,1600.0
sec3,30.0,420.0,240.0,350.0,110.0,1150.0
labor,500.0,250.0,330.0,NaN,NaN,NaN
capital,190.0,230.0,180.0,NaN,NaN,NaN
Total,1000.0,1600.0,1150.0,NaN,NaN,NaN


In [179]:
realq = Parameter(m, name="realq", description="real quantities", domain=[i,j])
realf = Parameter(m, name="realf", description="real factor quantities", domain=[x,j])
q0 = Parameter(m, name="q0", description="real total output", domain=[i])
a = Parameter(m, name="a", description="input-output coefficients", domain=[i,j])
b = Parameter(m, name="b", description="factor io coefficients", domain=[x,j])
p0 = Parameter(m, name="p0", description="initial price of commodities", domain=[i])
w0 = Parameter(m, name="w0", description="initial price of factors", domain=[x])
con = Parameter(m, name="con", description="consumption", domain=[i])
inv = Parameter(m, name="inv", description="investment", domain=[i])
lab = Parameter(m, name="lab", description="labor input", domain=[j])
cap = Parameter(m, name="cap", description="captial input", domain=[j])


In [180]:
# initial price
p0[i] = 1
w0[x] = 1

In [181]:
realq[i,j] = iotable[i,j]/p0[i]
realq.records.pivot(index="i",columns="j", values="value")

j,sec1,sec2,sec3
i,,,
sec1,200.0,300.0,150.0
sec2,80.0,400.0,250.0
sec3,30.0,420.0,240.0


In [182]:
realf[x,j] = iotable[x,j]/w0[x]
realf.records.pivot(index="x",columns="j",values="value")

j,sec1,sec2,sec3
x,,,
labor,500.0,250.0,330.0
capital,190.0,230.0,180.0


In [183]:
q0[i] = iotable[i,"Total"]/p0[i]
a[i,j] = realq[i,j]/q0[j]
a.records.pivot(index="i",columns="j",values="value")

j,sec1,sec2,sec3
i,,,
sec1,0.20,0.1875,0.130435
sec2,0.08,0.2500,0.217391
sec3,0.03,0.2625,0.208696


In [184]:
b[x,j] = realf[x,j]/q0[j]
b.records.pivot(index="x",columns="j",values="value")

j,sec1,sec2,sec3
x,,,
labor,0.50,0.15625,0.286957
capital,0.19,0.14375,0.156522


In [185]:
con[i] = iotable[i,"consumption"]/p0[i]
inv[i] = iotable[i,"investment"]/p0[i]
lab[j] = iotable["labor",j]/w0["labor"]
cap[j] = iotable["capital",j]/w0["capital"]

### variables

In [186]:
p = Variable(m, name="p", description="commodities price", domain=[i])
w = Variable(m, name="w", description="factor price", domain=[x])
dummy = Variable(m , name="dummy", description="dummy variable")

### equations

In [187]:
peq = Equation(m, name="peq", description="price equation", domain=[j])
peq[j] = Sum(i, a[i,j] * p[i]) + Sum(x, b[x,j]*w[x]) == p[j]

dummyeq = Equation(m, name="dummyeq", description="dummy equation")
dummyeq[...] = dummy == 0

pseq = Equation(m, name="pseq", domain=[s])
pseq[s] = Sum(i,a[i,s]*p[i]) +  Sum(x, b[x,s]*w[x]) == p[s]

### model

In [188]:
model0 = Model(m, name="model0", equations=[peq,dummyeq], sense="min", objective=dummy, problem="lp")
model1 = Model(m, name="model1", equations=[pseq,dummyeq], sense="min", objective=dummy, problem="lp")


### results

In [189]:
# 平衡价格
p.l[i] = 100
w.fx[x] = w0[x]
model0.solve()
p.records

,i,level,marginal,lower,upper,scale
0,sec1,1.0,0.0,-inf,inf,1.0
1,sec2,1.0,0.0,-inf,inf,1.0
2,sec3,1.0,0.0,-inf,inf,1.0


In [190]:
# 假设工资上涨20%
w1 = Parameter(m, name="w1", domain=[x],records=[["capital",1],["labor",1.2]])
p.l[i] = 100
w.fx[x] = w1[x]
model0.solve()
p.records

,i,level,marginal,lower,upper,scale
0,sec1,1.142117,0.0,-inf,inf,1.0
1,sec2,1.122565,0.0,-inf,inf,1.0
2,sec3,1.129625,0.0,-inf,inf,1.0


In [191]:
# 假设商品2价格上涨10%
p.fx["sec2"] = 1.1
w.fx[x] = w0[x]
model1.solve()
p.records

,i,level,marginal,lower,upper,scale
0,sec1,1.011099,0.0,-inf,inf,1.0
1,sec2,1.100000,-0.0,1.1,1.1,1.0
2,sec3,1.029302,0.0,-inf,inf,1.0
